# 네이버 웹툰 정보수집

###  #01.패키지 참조

기존의 request방식으로 크롤링하면 네이버 웹툰사이트 같이 Element 정보가 안나오는 경우가 있음 

Selenium
: 셀리늄은 웹브라우저를 제어할수 있는 패키지 
chrome_driver.exe
: 셀레늄과 크롬브라우저를 연결 chromedriver_autoinstaller 페키지필요
: 파이선 스스토 크럼드라이버를 내려받도록 하기 위해 
    -> 셀리늄은 1) 특정페이지 접속
                2) Element 코드 가죠옴
                3) 특정 요소에 키보드 입력 보내기 
                4) 마우스 클릭 보내기 
                5) 자바스크립크 실행 -> 스크롤 발생시킴


    -> webdriverwait : 크롬 제어기
    -> time : 지정된 시간동안 랙을 거는 기능 


    

요일별 웹툰 -> 한 날짜에서 웹툰별 이미지 링크 -> 웹툰정보 + 웹툰 주소로 이동하는 링크 + 제목 + 작가 + 별점

In [2]:
# 필요한 모듈 참조

## chromedriver_autoinstaller 자동 설치 모듈
import chromedriver_autoinstaller
## 크롬을 제어하기 위한 객체
from selenium import webdriver

## 크롬이 웹페이지 로딩을 완료할 떄 까지 최대 n초간 대기하는 기능
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup
from pandas import DataFrame
import time

#### #02.크롬브라우저 가동




In [6]:
## 크롬드라이버 자동 ㅊ치
chromedriver_autoinstaller.install()

## 크롬드라이버를 통해 크롬을 실행시킴
##  -> driver 객체는 chrome 자체

driver = webdriver.Chrome()
## 크롬 브라우저가 준비 될떄 까지 최대 5초 대기 
driver.implicitly_wait(5)


In [8]:
#  요일별 네이버 웹툰에 접근하기 위한 변수값
# -> url을 분석하여 얻어낸 값 

params = ['mon','tue','wed','thu','fri','sat','sun','dailyPlus']

## 네이버 웹툰의 주소형식 

naverWebtoonUrl = "https://comic.naver.com/webtoon?tab={0}"



#### #03. 응답결과로부터 내용 추출하기

In [28]:
## 수집된 결과가 누적될 빈 리스트 

naverWebtoonList = []

## 요일별 반복 
for p in params :
    ## 특정 요일의 네이버 웹툰페이지 
    url = naverWebtoonUrl.format(p)
    # print(url)

    ## 크롬브라우저로 방문
    driver.get(url)
    time.sleep(1)
    ## 브라우저에 표시되는 전체 html 코드 추출
    soup = BeautifulSoup(driver.page_source)
    # print(soup)
    
    # 전체 웹툰이 표시되는 부분에서 
    
    webtoonList = soup.select(".ContentList__content_list--q5KXY > .item")

    # print(webtoonList)
    for w in webtoonList :
        # 포스터 url
        posterEl = w.select(".Poster__image--d9XTI")

        ## 가져온 이미지가 존재하고 src 존재 여부가 중요
        if posterEl and "src" in posterEl[0].attrs:
            poster = posterEl[0].attrs['src']
        else :
            None
        # print(poster)
        
        # 포스터 링크안에 있는 웹툰의 url 가져오기 
        urlEl = w.select(".Poster__link--sopnC")
        # print(urlEl)
        # url이 존재하고, href가 존재해야함.
        if urlEl and "href" in urlEl[0].attrs :
            url = urlEl[0].attrs['href']
            ## 외부사이트에서 접속하는 경우에는 full url 필요
            if url.find("https://comic.naver.com") == -1 :
                url = "https://comic.naver.com" + url
        else : 
            url = None
        
        # 웹툰제목가져오기 

        titleEl = w.select(".ContentTitle__title--e3qXt > .text")
        # print(titleEl)
        if titleEl :
            title = titleEl[0].text.strip()
        else : 
            title = None
        # 작가 이름 가져오기 
        
        authorEl = w.select(" .ContentAuthor__author--CTAAP")
        # print(authorEl)
        if authorEl :
            author = authorEl[0].text.strip()
        else : 
            author = None
        # print(author)

        ## 별점가져오기 
        ratingEl = w.select(".Rating__ico_star--BcfMb > .text")
        if ratingEl :
            rating = ratingEl[0].text.strip()
        else : 
            rating = None

        ## 결과 병합 
        result_dict = {"요일" :p , "제목" : title,"작가" :author ,"별점" : rating}
        # print(result_dict)
        naverWebtoonList.append(result_dict)
df = DataFrame(naverWebtoonList)
df.to_excel("요일별_네이버웹툰.xlsx")
df



,요일,제목,작가,별점
0,mon,뷰티풀 군바리,설이 / 윤성원,None
1,mon,신의 탑,SIU,None
2,mon,퀘스트지상주의,박태준 만화회사,None
3,mon,똑 닮은 딸,이담,None
4,mon,백수세끼,치즈,None
...,...,...,...,...
721,dailyPlus,후드,후렛샤 / 심규호,None
722,dailyPlus,실버벨,노떼,None
723,dailyPlus,씨앗의 정원,백희,None
724,dailyPlus,도와줘우주,설레임 / 한겨울,None


,요일,제목,작가,별점
0,dailyPlus,[드라마원작] 사냥개들,정찬,None
1,dailyPlus,원룸,휘랭,None
2,dailyPlus,발칙한 춘향,이Zen / Jean / 금은정,None
3,dailyPlus,주체할 수 없는,Guoxiao / Zero,None
4,dailyPlus,지옥락,Yuji Kaku,None
...,...,...,...,...
122,dailyPlus,후드,후렛샤 / 심규호,None
123,dailyPlus,실버벨,노떼,None
124,dailyPlus,씨앗의 정원,백희,None
125,dailyPlus,도와줘우주,설레임 / 한겨울,None
